### 01. 라이브러리 및 데이터 호출

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import LSTM, Dense
import matplotlib.pyplot as plt

In [2]:
train = pd.read_csv('data/train.csv')
submission = pd.read_csv('data/submission.csv')

--------------------------

### 02. 데이터 기초 정보 확인

In [3]:
# 데이터 처음,끝부분 확인
head = train.head(5)
tail = train.head(5)

# Columns 확인
columns = train.columns

# Data Shape 확인
shape = train.shape

# 통계값 확인
describe = train.describe()

# Columns Type 확인
dtypes = train.dtypes

-----------------------

### 03. 데이터 전처리

#### 3-1. 결측치 확인
###### [1] 결측치와 중복되는 행이 모두 없음을 확인할 수 있다
###### [2] 따라서 결측치 처리에 대한 부분이 불필요

In [4]:
# 컬럼별 결측치 파악
null = train.isna().sum()

# 중복되는 행 파악
duplicated = train[train.duplicated()]

#### 3-2. 시간 데이터 전처리
###### [1] 시간 기반 데이터는 Time 컬럼을 인덱스로 설정하면 시간 시계열 분석에 유용

In [5]:
# 01. time 컬럼을 datetime 형식으로 변환
train['Time'] = pd.to_datetime(train['Time'])

# 02. 년, 월, 일, 시, 분, 초 등의 특성 추출
train['year'] = train['Time'].dt.year
train['month'] = train['Time'].dt.month
train['day'] = train['Time'].dt.day
train['hour'] = train['Time'].dt.hour
train['minute'] = train['Time'].dt.minute
train['second'] = train['Time'].dt.second

# 03. 요일 추출 (0=월요일, 6=일요일)
train['weekday'] = train['Time'].dt.weekday

# 04. 주차 추출
train['week'] = train['Time'].dt.isocalendar().week
